In [74]:
import re
import pickle
from datetime import datetime
from collections import Counter
from bs4 import BeautifulSoup

In [90]:
with open('data/export_19/messages.html') as f:
    soup = BeautifulSoup(f)

soups = [soup]
for i in range(2,48):
    with open('data/export_19/messages{}.html'.format(i)) as f:
        soups.append(BeautifulSoup(f))

In [91]:
replies_all = []
for s in soups:
    replies_all.append(s.find_all('div', 'reply_to'))

In [71]:
reply_nw = []
for i, rs in enumerate(replies_all):
    for r in rs:
        frm = r.parent.find('div', class_='from_name')
        if frm is None:
            continue
        frm = frm.text.strip()
        if 'via @' in frm:
            frm = frm.split('via')[0].strip()
        time = datetime.strptime(r.parent.find(class_='date')['title'], '%d.%m.%Y %H:%M:%S').isoformat()
        to_msg = r.a['href'].split('_')[-1]
        for j in range(i, -1, -1):
            to = soups[j].find('div', id=to_msg)
            if (to is not None):
                break
        if to is None:
            continue
        to_name = to.find('div', class_='from_name')
        if to_name is None:
            continue
        to_name = to_name.text.strip()
        if 'via @' in to_name:
            to_name = to_name.split('via')[0].strip()
        reply_nw.append((frm, to_name, time))

In [72]:
pickle.dump(reply_nw, open('data/edges_w_timestamps.p', 'wb'))

In [ ]:
edge_counts = Counter(reply_nw)
edges = []
for e, v in edge_counts.items():
    edges.append((e[0], e[1], {'weight': v/msg_counts[e[0]]}))

In [ ]:
pickle.dump(reply_nw, open('data/weighted_edges.p', 'wb'))

In [13]:
users = []
for s in soups:
    for d in s.find_all('span', class_='details'):
        d.decompose()
    names = s.find_all('div', class_='from_name')
    for n in names:
        n = n.text.strip()
        if 'via @' in n:
            n = n.split('via')[0].strip()
        users.append(n)
msg_counts = Counter(users)

In [92]:
dates = []
for s in soups:
    names = s.find_all('div', class_='from_name')
    for n in names:
        if n.parent is None:
            print('asd')
            continue
        if n.parent.find(class_='date') is None: continue
        date = datetime.strptime(n.parent.find(class_='date')['title'], '%d.%m.%Y %H:%M:%S').date()
        dates.append(date)

In [95]:
msg_counts_19 = Counter(dates)
pickle.dump(msg_counts_19, open('data/msg_counts_date_19.p', 'wb'))

In [7]:
pickle.dump(msg_counts, open('data/msg_counts.p', 'wb'))